## Домашнее задание №3

##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

In [ ]:
import numpy as np
import time

import torch
from torch import nn
from torch.nn import functional as F

import torchvision
from torchvision.datasets import MNIST

from matplotlib import pyplot as plt
from IPython.display import clear_output

### Задача №1: 
Вернемся к задаче распознавания рукописных цифр, рассмотренной на первом занятии. Все также будем работать с набором данных [MNIST](http://yann.lecun.com/exdb/mnist/). В данном задании воспользуемся всем датасетом целиком.

__Ваша основная задача: реализовать весь пайплан обучения модели и добиться качества $\geq 92\%$ на тестовой выборке.__

Код для обучения модели в данном задании отсутствует. Присутствует лишь несколько тестов, которые помогут вам отладить свое решение. За примером можно обратиться к ноутбуку первого занятия.

Настоятельно рекомендуем написать код "с нуля", лишь поглядывая на пример, а не просто "скопировать-вставить". Это поможет вам в дальнейшем.

In [ ]:
# do not change the code in the block below
# __________start of block__________

train_mnist_data = MNIST('.', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_mnist_data = MNIST('.', train=False, transform=torchvision.transforms.ToTensor(), download=True)


train_data_loader = torch.utils.data.DataLoader(
    train_mnist_data,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

test_data_loader = torch.utils.data.DataLoader(
    test_mnist_data,
    batch_size=32,
    shuffle=False,
    num_workers=2
)

random_batch = next(iter(train_data_loader))
_image, _label = random_batch[0][0], random_batch[1][0]
plt.figure()
plt.imshow(_image.reshape(28, 28))
plt.title(f'Image label: {_label}')
# __________end of block__________

Постройте модель ниже. Пожалуйста, не стройте переусложненную сеть, не стоит делать ее глубже четырех слоев (можно и меньше). Ваша основная задача – обучить модель и получить качество на отложенной (тестовой выборке) не менее 92% accuracy.

*Комментарий: для этого достаточно линейных слоев и функций активации.*

__Внимание, ваша модель должна быть представлена именно переменной `model`.__

In [ ]:
model = nn.Sequential(
    nn.Linear(784, 200),
    nn.ReLU(),
    nn.Linear(200, 10),
    nn.Sigmoid()
)

opt = torch.optim.Adam(model.parameters(), lr=1e-3)

Локальные тесты для проверки вашей модели доступны ниже:

In [ ]:
# do not change the code in the block below
# __________start of block__________
assert model is not None, 'Please, use `model` variable to store your model'

try:
    x = random_batch[0].reshape(-1, 784)
    y = random_batch[1]

    # compute outputs given inputs, both are variables
    y_predicted = model(x)    
except Exception as e:
    print('Something is wrong with the model')
    raise e
    
    
assert y_predicted.shape[-1] == 10, 'Model should predict 10 logits/probas'

print('Everything seems fine!')
# __________end of block__________

Настройте параметры модели на обучающей выборке. В качестве примера можете воспользоваться ноутбуком с занятия №1. Также рекомендуем поработать с различными оптимизаторами.

In [ ]:
def train_model(model, train_data_loader, val_loader, opt, n_epochs: int):
    train_loss = []
    val_loss = []
    val_accuracy = []
    
    # Use CrossEntropyLoss for multi-class classification
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(n_epochs):
        time_start = time.time()
        ep_train_loss = []
        ep_val_loss = []
        ep_val_accuracy = []

        # Training phase
        model.train()
        for x_batch, y_batch in train_data_loader:
            # Flatten the input (assuming MNIST-like 28x28 images)
            x_batch = x_batch.view(-1, 784)
            
            preds = model(x_batch)
            loss = loss_fn(preds, y_batch)  # CrossEntropyLoss handles class indices

            opt.zero_grad()
            loss.backward()
            opt.step()

            ep_train_loss.append(loss.item())

        time_end = time.time()

        # Validation phase
        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                x_batch = x_batch.view(-1, 784)
                preds = model(x_batch)
                loss = loss_fn(preds, y_batch)

                ep_val_loss.append(loss.item())
                # Calculate accuracy
                _, predicted = torch.max(preds.data, 1)
                correct = (predicted == y_batch).sum().item()
                ep_val_accuracy.append(correct / y_batch.size(0))

        # Print statistics
        epoch_train_loss = np.mean(ep_train_loss)
        epoch_val_loss = np.mean(ep_val_loss)
        epoch_val_acc = np.mean(ep_val_accuracy)
        
        print(f'Epoch {epoch+1}/{n_epochs} took {time_end - time_start:.2f} seconds')
        print(f'Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.4f}')

        # Store metrics
        train_loss.append(epoch_train_loss)
        val_loss.append(epoch_val_loss)
        val_accuracy.append(epoch_val_acc)
        assert epoch_val_acc >= 0.92, 'Test accuracy is below 0.92 threshold'
        
    return train_loss, val_loss, val_accuracy

Также, напоминаем, что в любой момент можно обратиться к замечательной [документации](https://pytorch.org/docs/stable/index.html) и [обучающим примерам](https://pytorch.org/tutorials/).  

Оценим качество классификации:

In [ ]:
# predicted_labels = []
# real_labels = []
# model.eval()
# with torch.no_grad():
#     for batch in train_data_loader:
#         y_predicted = model(batch[0].reshape(-1, 784))
#         predicted_labels.append(y_predicted.argmax(dim=1))
#         real_labels.append(batch[1])

# predicted_labels = torch.cat(predicted_labels)
# real_labels = torch.cat(real_labels)
# train_acc = (predicted_labels == real_labels).type(torch.FloatTensor).mean()

In [ ]:
# print(f'Neural network accuracy on train set: {atrain_acc:3.5}')

In [ ]:
# predicted_labels = []
# real_labels = []
# model.eval()
# with torch.no_grad():
#     for batch in test_data_loader:
#         y_predicted = model(batch[0].reshape(-1, 784))
#         predicted_labels.append(y_predicted.argmax(dim=1))
#         real_labels.append(batch[1])

# predicted_labels = torch.cat(predicted_labels)
# real_labels = torch.cat(real_labels)
# test_acc = (predicted_labels == real_labels).type(torch.FloatTensor).mean()

In [ ]:
# print(f'Neural network accuracy on test set: {test_acc:3.5}')

Проверка, что необходимые пороги пройдены:

In [ ]:
assert test_acc >= 0.92, 'Test accuracy is below 0.92 threshold'
assert train_acc >= 0.91, 'Train accuracy is below 0.91 while test accuracy is fine. We recommend to check your model and data flow'

### Сдача задания
Загрузите файл `hw03_data_dict.npy` (ссылка есть на странице с заданием) и запустите код ниже для генерации посылки. Код ниже может его загрузить (но в случае возникновения ошибки скачайте и загрузите его вручную).

In [ ]:
!wget https://raw.githubusercontent.com/girafe-ai/ml-course/msu_branch/homeworks/hw03_mnist/hw03_data_dict.npy

In [ ]:
from posixpath import sep
import json
# do not change the code in the block below
# __________start of block__________
import os

assert os.path.exists('hw03_data_dict.npy'), 'Please, download `hw03_data_dict.npy` and place it in the working directory'

def get_predictions(model, eval_data, step=10):

    predicted_labels = []
    model.eval()
    with torch.no_grad():
        for idx in range(0, len(eval_data), step):
            y_predicted = model(eval_data[idx:idx+step].reshape(-1, 784))
            predicted_labels.append(y_predicted.argmax(dim=1))

    predicted_labels = torch.cat(predicted_labels)
    return predicted_labels

loaded_data_dict = np.load('hw03_data_dict.npy', allow_pickle=True)

submission_dict = {
    'train': ','.join(str(_) for _ in get_predictions(model, torch.FloatTensor(loaded_data_dict.item()['train'])).numpy().tolist()),
    'test': ','.join(str(_) for _ in get_predictions(model, torch.FloatTensor(loaded_data_dict.item()['test'])).numpy().tolist())
    # 'train': get_predictions(model, torch.FloatTensor(loaded_data_dict.item()['train'])).numpy(),
    # 'test': get_predictions(model, torch.FloatTensor(loaded_data_dict.item()['test'])).numpy()
}

file_path = 'submission_dict_hw03.npy'
json.dump(submission_dict, open(file_path, 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4)

# np.save('submission_dict_hw03.npy', submission_dict, allow_pickle=True)
print('File saved to `submission_dict_hw03.npy`')
# __________end of block__________

На этом задание завершено. Поздравляем!